# Google Colab setup with Google Drive folder

This notebook provides the code you need to set up Google Colab to run and import files from within a Google Drive folder.

This will allow you to upload assignment code to your Google Drive and then run the code on Google Colab machines (with free GPUs if needed). 

You will need to create a folder in your Google Drive to hold your assignments and you will need to open Colaboratory within this folder before running the set up code (check the link above to see how).

Note: this use of Google Drive is optional, and you could also just manually copy the data into your colab runtime. Keep in mind, this won't be persistent though, and you will have to download your models / plots before the runtime shuts down.

# Mount Google Drive

This will allow the Colab machine to access Google Drive folders by mounting the drive on the machine. You will be asked to copy and paste an authentication code.

In [1]:
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [2]:
ls

gdrive/  sample_data/


# Change directory to allow imports


As noted above, you should create a Google Drive folder to hold all your assignment files. You will need to add this code to the top of any python notebook you run to be able to import python files from your drive assignment folder (you should change the file path below to be your own assignment folder).

In [0]:
import os
os.chdir("/content/gdrive/My Drive/CS_543_MP4")

In [4]:
ls # Check if this is your MP4 folder

data/  plot.png


# Download data into your data folders

Within the folder CS_543_MP4, create a subfolder called 'data'. Copy all the files in this Google drive link(https://drive.google.com/drive/folders/18wgGNdbkptqnb8W8Ao_0rQ0o-GQdHils?usp=sharing) to your data folder. Please do not change the file structures.

# Set up GPU and PyTorch

First, ensure that your notebook on Colaboratory is set up to use GPU. After opening the notebook on Colaboratory, go to Edit>Notebook settings, select Python 3 under "Runtime type," select GPU under "Hardware accelerator," and save.

Next, install PyTorch:

In [5]:
!pip3 install torch torchvision

Make sure that pytorch is installed and works with GPU:

In [6]:
import torch
a = torch.Tensor([1]).cuda()
print(a)


tensor([1.], device='cuda:0')


In [7]:
torch.cuda.is_available()

True

# Part 1

In [8]:
"""Headers"""

from __future__ import print_function
from PIL import Image
import os
import os.path
import numpy as np
import sys
if sys.version_info[0] == 2:
    import cPickle as pickle
else:
    import pickle

import torch.utils.data as data
from torchvision.datasets.utils import download_url, check_integrity
import copy
import csv
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import os.path
from tqdm.notebook import tqdm
import sys
import torch
import torch.utils.data
import torchvision
import torchvision.transforms as transforms

from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

np.random.seed(111)
torch.cuda.manual_seed_all(111)
torch.manual_seed(111)


Training an image classifier
----------------------------

We will do the following steps in order:

1. Load the randomized FashionMNIST training, validation and test datasets using
   torchvision. Use torchvision.transforms to apply transforms on the
   dataset.
2. Define a Convolution Neural Network - BaseNet
3. Define a loss function and optimizer
4. Train the network on training data and check performance on val set.
   Plot train loss and validation accuracies.
5. Try the network on test data and create .csv file for submission to kaggle

In [0]:
# <<TODO#5>> Based on the val set performance, decide how many
# epochs are apt for your model.
# ---------
EPOCHS = 30
# ---------

IS_GPU = True
TEST_BS = 256
TOTAL_CLASSES = 10
TRAIN_BS = 32
PATH_TO_FASHIONMNIST_CS543 = "data/fashionMNIST/"

In [0]:
def calculate_accuracy(dataloader, is_gpu):
    """ Util function to calculate val set accuracy,
    both overall and per class accuracy
    Args:
        dataloader (torch.utils.data.DataLoader): val set 
        is_gpu (bool): whether to run on GPU
    Returns:
        tuple: (overall accuracy, class level accuracy)
    """    
    correct = 0.
    total = 0.
    predictions = []

    class_correct = list(0. for i in range(TOTAL_CLASSES))
    class_total = list(0. for i in range(TOTAL_CLASSES))

    for data in dataloader:
        images, labels = data
        if is_gpu:
            images = images.cuda()
            labels = labels.cuda()
        outputs = net(Variable(images))
        _, predicted = torch.max(outputs.data, 1)
        predictions.extend(list(predicted.cpu().numpy()))
        total += labels.size(0)
        correct += (predicted == labels).sum()

        c = (predicted == labels).squeeze()
        for i in range(len(labels)):
            label = labels[i]
            class_correct[label] += c[i]
            class_total[label] += 1

    class_accuracy = 100 * np.divide(class_correct, class_total)
    return 100*correct/total, class_accuracy

1.**Loading FashionMNIST_CS543**

We modify the dataset to create FashionMNIST_CS543 dataset which consist of 60000 training images, 5000 validation images and 5000 test images. The train and val datasets have labels while all the labels in the test set are set to 0.


In [11]:
# The output of torchvision datasets are PILImage images of range [0, 1].
# Using transforms.ToTensor(), transform them to Tensors of normalized range
# [-1, 1].


# <<TODO#1>> Use transforms.Normalize() with the right parameters to 
# make the data well conditioned (zero mean, std dev=1) for improved training.
# <<TODO#2>> Try using transforms.RandomCrop() and/or transforms.RandomHorizontalFlip()
# to augment training data.
# After your edits, make sure that test_transform should have the same data
# normalization parameters as train_transform
# You shouldn't have any data augmentation in test_transform (val or test data is never augmented).
# ---------------------
# notice for some transform like transforms.RandomRotation, given we are dealing with grayscale input, you need to put fill=(0,) in the parameter. 
train_transform = transforms.Compose([
  transforms.RandomHorizontalFlip(),
  # transforms.RandomVerticalFlip(),
  transforms.RandomCrop(size=28, padding=1),
  transforms.ToTensor(),
  transforms.Normalize((0,), (1,))])
test_transform = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Normalize((0,), (1,))])
# ---------------------

#DO NOT CHANGE any line below
train_dataset = torchvision.datasets.FashionMNIST(root = "data", train=True, transform=train_transform, target_transform=None, download=False) 
test_dataset = torchvision.datasets.FashionMNIST(root = "data", train=False, transform=test_transform, target_transform=None, download=False)
val_dataset = copy.deepcopy(test_dataset)
val_dataset.data = torch.load("data/FashionMNIST/processed/val.pt")[0]
val_dataset.targets = torch.load("data/FashionMNIST/processed/val.pt")[1]
print("train_dataset data shape: ", train_dataset.data.shape)
print("train_dataset labels shape: ", train_dataset.targets.shape)
print()
print("val_dataset data shape: ", val_dataset.data.shape)
print("val_dataset labels shape",val_dataset.targets.shape)

# check for Dataloader function: https://pytorch.org/docs/stable/data.html
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=TRAIN_BS, shuffle=True, num_workers=2, drop_last=True)  #DO NOT CHANGE
valloader = torch.utils.data.DataLoader(val_dataset, batch_size=TEST_BS, shuffle=False, num_workers=2, drop_last=False) #DO NOT CHANGE
testloader = torch.utils.data.DataLoader(test_dataset, batch_size=TEST_BS, shuffle=False, num_workers=2, drop_last=False) #DO NOT CHANGE

# The 10 classes for FashionMNIST
classes = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat", "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]

train_dataset data shape:  torch.Size([60000, 28, 28])
train_dataset labels shape:  torch.Size([60000])

val_dataset data shape:  torch.Size([5000, 28, 28])
val_dataset labels shape torch.Size([5000])


In [0]:
########################################################################
# 2. Define a Convolution Neural Network
# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
# We provide a basic network that you should understand, run and
# eventually improve
# <<TODO>> Add more conv layers
# <<TODO>> Add more fully connected (fc) layers
# <<TODO>> Add regularization layers like Batchnorm.
#          nn.BatchNorm2d after conv layers:
#          http://pytorch.org/docs/master/nn.html#batchnorm2d
#          nn.BatchNorm1d after fc layers:
#          http://pytorch.org/docs/master/nn.html#batchnorm1d
# This is a good resource for developing a CNN for classification:
# http://cs231n.github.io/convolutional-networks/#layers

import torch.nn as nn
import torch.nn.functional as F

class BaseNet(nn.Module):
    def __init__(self):
        super(BaseNet, self).__init__()
        
        # <<TODO#3>> Add more conv layers with increasing 
        # output channels
        # <<TODO#4>> Add normalization layers after conv
        # layers (nn.BatchNorm2d)

        # Also experiment with kernel size in conv2d layers (say 3
        # inspired from VGGNet)
        # To keep it simple, keep the same kernel size
        # (right now set to 5) in all conv layers.
        # Do not have a maxpool layer after every conv layer in your
        # deeper network as it leads to too much loss of information.

        self.conv1 = nn.Conv2d(1, 6, 5)
        self.BH1 = nn.BatchNorm2d(num_features=6)
        self.pool = nn.MaxPool2d(2, 2)

        self.conv2 = nn.Conv2d(6, 16, 5)
        self.BH2 = nn.BatchNorm2d(num_features=16)


        self.conv3 = nn.Conv2d(16, 32, 2)
        self.BH3 = nn.BatchNorm2d(num_features=32)



        # <<TODO#3>> Add more linear (fc) layers
        # <<TODO#4>> Add normalization layers after linear and
        # experiment inserting them before or after ReLU (nn.BatchNorm1d)
        # More on nn.sequential:
        # http://pytorch.org/docs/master/nn.html#torch.nn.Sequential
        
        self.fc_net = nn.Sequential(
            nn.Linear(32 * 3 * 3, 50),
            nn.BatchNorm1d(num_features=50),
            nn.ReLU(inplace=True),
            nn.Linear(50, TOTAL_CLASSES),
            nn.BatchNorm1d(num_features=TOTAL_CLASSES),
        )

    def forward(self, x):

        # <<TODO#3&#4>> Based on the above edits, you'll have
        # to edit the forward pass description here.
        # print('1',x.shape)
        x=self.BH1(self.conv1(x))
        x = self.pool(F.relu(x))
        # Output size = 24//2 x 24//2 = 12 x 12
        # print('2',x.shape)
        x = self.BH2(self.conv2(x))
        x = self.pool(F.relu(x))
        # Output size = 8//2 x 8//2 = 4 x 4
        # print('3',x.shape)

        x = self.BH3(self.conv3(x))
        x = F.relu(x)
        # print('4',x.shape)
        # Output size = 3 * 3 * 32

        # See the CS231 link to understand why this is 16*5*5!
        # This will help you design your own deeper network
        x = x.view(-1, 32 * 3 * 3)
        # print('5',x.shape)
        x = self.fc_net(x)
        # print('6',x.shape)
        # No softmax is needed as the loss function in step 3
        # takes care of that
        
        return x

# Create an instance of the nn.module class defined above:
net = BaseNet()

# For training on GPU, we need to transfer net and data onto the GPU
# http://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html#training-on-gpu
if IS_GPU:
    net = net.cuda()


In [14]:
print(net)

BaseNet(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (BH1): BatchNorm2d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (BH2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(16, 32, kernel_size=(2, 2), stride=(1, 1))
  (BH3): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc_net): Sequential(
    (0): Linear(in_features=288, out_features=50, bias=True)
    (1): BatchNorm1d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Linear(in_features=50, out_features=10, bias=True)
    (4): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
)


In [0]:
########################################################################
# 3. Define a Loss function and optimizer
# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
# Here we use Cross-Entropy loss and SGD with momentum.
# The CrossEntropyLoss criterion already includes softmax within its
# implementation. That's why we don't use a softmax in our model
# definition.

import torch.optim as optim
criterion = nn.CrossEntropyLoss()

# Tune the learning rate.
# See whether the momentum is useful or not
optimizer = optim.SGD(net.parameters(), lr=0.005, momentum=0.9)
# optimizer = torch.optim.Adam(net.parameters(), lr=0.005)

plt.ioff()
fig = plt.figure()
train_loss_over_epochs = []
val_accuracy_over_epochs = []


In [16]:
########################################################################
# 4. Train the network
# ^^^^^^^^^^^^^^^^^^^^
#
# We simply have to loop over our data iterator, and feed the inputs to the
# network and optimize. We evaluate the validation accuracy at each
# epoch and plot these values over the number of epochs
# Nothing to change here
# -----------------------------
for epoch in tqdm(range(EPOCHS), total=EPOCHS):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data

        if IS_GPU:
            inputs = inputs.cuda()
            labels = labels.cuda()

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        # print(outputs.shape,labels.shape)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
    
    # Normalizing the loss by the total number of train batches
    running_loss/=len(trainloader)
    print('[%d] loss: %.3f' %
          (epoch + 1, running_loss))

    # Scale of 0.0 to 100.0
    # Calculate validation set accuracy of the existing model
    val_accuracy, val_classwise_accuracy = \
        calculate_accuracy(valloader, IS_GPU)
    print('Accuracy of the network on the val images: %d %%' % (val_accuracy))

    # # Optionally print classwise accuracies
    # for c_i in range(TOTAL_CLASSES):
    #     print('Accuracy of %5s : %2d %%' % (
    #         classes[c_i], 100 * val_classwise_accuracy[c_i]))

    train_loss_over_epochs.append(running_loss)
    val_accuracy_over_epochs.append(val_accuracy)
# -----------------------------


# Plot train loss over epochs and val set accuracy over epochs
# Nothing to change here
# -------------
plt.subplot(2, 1, 1)
plt.ylabel('Train loss')
plt.plot(np.arange(EPOCHS), train_loss_over_epochs, 'k-')
plt.title('train loss and val accuracy')
plt.xticks(np.arange(EPOCHS, dtype=int))
plt.grid(True)

plt.subplot(2, 1, 2)
plt.plot(np.arange(EPOCHS), val_accuracy_over_epochs, 'b-')
plt.ylabel('Val accuracy')
plt.xlabel('Epochs')
plt.xticks(np.arange(EPOCHS, dtype=int))
plt.grid(True)
plt.savefig("plot.png")
plt.close(fig)
print('Finished Training')
# -------------

[1] loss: 0.638
Accuracy of the network on the val images: 85 %
[2] loss: 0.465
Accuracy of the network on the val images: 86 %
[3] loss: 0.419
Accuracy of the network on the val images: 88 %
[4] loss: 0.393
Accuracy of the network on the val images: 88 %
[5] loss: 0.376
Accuracy of the network on the val images: 87 %
[6] loss: 0.363
Accuracy of the network on the val images: 89 %
[7] loss: 0.355
Accuracy of the network on the val images: 89 %
[8] loss: 0.341
Accuracy of the network on the val images: 88 %
[9] loss: 0.336
Accuracy of the network on the val images: 88 %
[10] loss: 0.330
Accuracy of the network on the val images: 89 %
[11] loss: 0.324
Accuracy of the network on the val images: 89 %
[12] loss: 0.320
Accuracy of the network on the val images: 88 %
[13] loss: 0.316
Accuracy of the network on the val images: 89 %
[14] loss: 0.313
Accuracy of the network on the val images: 89 %
[15] loss: 0.308
Accuracy of the network on the val images: 90 %
[16] loss: 0.303
Accuracy of the n

In [17]:
########################################################################
# 5. Try the network on test data, and report your performance in the submission report
# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
########################################################################

# Check out why .eval() is important!
# https://discuss.pytorch.org/t/model-train-and-model-eval-vs-model-and-model-eval/5744/2
net.eval()

test_accuracy, test_classwise_accuracy = \
        calculate_accuracy(testloader, IS_GPU)
print('Accuracy of the network on the test images: %d %%' % (test_accuracy))


Accuracy of the network on the test images: 91 %
